In [ ]:
import pandas as pd
from PIL import Image
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Input, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import plot_model
# Define train, test, and validation directories
train_dir = r'C:\Users\Microsoft\Money\dataset\test'
val_dir = r'C:\Users\Microsoft\Money\dataset\valid'
test_dir = r'C:\Users\Microsoft\Money\dataset\test'

In [ ]:
class_names = sorted([dir_name for dir_name in os.listdir(train_dir)])

# Define a smaller image size
IMG_SIZE = 128

In [ ]:
def read_images_from_folder(folder_path):
    images = []
    labels = []
    
    for class_folder in tqdm(os.listdir(folder_path)):
        class_path = os.path.join(folder_path, class_folder)
        if os.path.isdir(class_path):
            label = class_folder
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                img = Image.open(image_path).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
                img_array = np.array(img, dtype=np.uint8)
                images.append(img_array)
                labels.append(label)
    
    return np.array(images), np.array(labels)

In [ ]:
train_images, train_labels  = read_images_from_folder(train_dir)
valid_images, valid_labels  = read_images_from_folder(val_dir)
test_images, test_labels = read_images_from_folder(test_dir)

num_classes = len(np.unique(train_labels))
Classes = np.unique(train_labels)
print(f"Classes: {Classes}")
print(f"Number of classes: {num_classes}")

In [ ]:
def image_statistics(images, dataset_name):
    print(f"Dataset: {dataset_name}")
    print(f"Number of images: {len(images)}")
    
    # Calculate statistics about image shapes
    print(f"Image shapes: {images.shape}")
    print("------------------------------")

In [ ]:
# Example usage:
image_statistics(train_images, "Train")
image_statistics(valid_images, "Validation")
image_statistics(test_images, "Test")


plt.figure(figsize=(12, 6))
pd.DataFrame(train_labels).value_counts().sort_index().plot(kind='bar')
plt.title('Training Data Class Distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

In [ ]:
# Generate 9 random indices
sample_indices = random.sample(range(len(train_images)), 9)

# Create a 3x3 grid of subplots for plotting the images
plt.figure(figsize=(10, 10))
plt.suptitle("Some Sample Images", fontsize=16)

for i, idx in enumerate(sample_indices):
    plt.subplot(3, 3, i + 1)
    plt.imshow(train_images[idx])  # Assuming images are already in the appropriate format
    plt.title(f"Label: {train_labels[idx]}")
    plt.axis('off')

plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)
test_labels_encoded = label_encoder.transform(test_labels)

num_classes = len(label_encoder.classes_)

train_labels_one_hot = to_categorical(train_labels_encoded, num_classes=num_classes)
valid_labels_one_hot = to_categorical(valid_labels_encoded, num_classes=num_classes)
test_labels_one_hot = to_categorical(test_labels_encoded, num_classes=num_classes)

In [ ]:
def plot_history(history):   
    # Access the training history
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    # Create subplots for loss and accuracy
    plt.figure(figsize=(12, 4))
    # Plot training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    plt.tight_layout()
    plt.show()

In [ ]:
def predict_class(model, test_images, test_labels, num_images_to_plot=12, num_columns=4):
    # Decode one-hot encoded labels
    test_labels = np.argmax(test_labels_one_hot, axis=1)
    
    # Choose random indices for testing
    random_indices = np.random.choice(len(test_images), size=num_images_to_plot, replace=False)

    # Calculate the number of rows needed based on the number of images and columns
    num_rows = int(np.ceil(len(random_indices) / num_columns))

    # Create a subplot with the specified number of rows and columns
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 3*num_rows))

    # Loop through the selected indices for prediction and plotting
    for i, ax in zip(random_indices, axes.flatten()):
        # Get the image and label
        image = test_images[i]
        label = test_labels[i]

        # Reshape the image to match the input shape expected by the model
        image = np.expand_dims(image, axis=0)

        # Predict the image using the loaded model
        prediction = model.predict(image)

        # Get the predicted label
        predicted_label = np.argmax(prediction)

        # Plot the image
        ax.imshow(image.squeeze())  # Squeeze to remove the singleton dimension
        ax.set_title(f"Actual Label: {label}\n Predicted Label: {predicted_label}")

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()

In [ ]:
# Define the number of epochs
NUM_CLASSES = num_classes
num_epochs = 200
batch_size = 32

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=30, verbose=2, restore_best_weights=True)

# Build the deep learning model
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()
#plot_model(model, show_shapes=True, show_layer_names=True)

# Start training the model
history = model.fit(
    train_images,
    train_labels_one_hot,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_data=(valid_images, valid_labels_one_hot),
    callbacks=[early_stopping]
)
plot_history(history)
model.evaluate(test_images, test_labels_one_hot)
predict_class(model, test_images, test_labels_one_hot)

In [ ]:
# save model
model.save('model.h5')
from tensorflow.keras.applications import MobileNetV2

In [ ]:
# Use MobileNetV2 as an example of a lighter architecture
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')

model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.evaluate(test_images, test_labels_one_hot)